# Introductory cell

 first we load the scaled data as saved in the preprocessing and engineering notebook 

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the data
X_train_scaled = pd.read_csv("../data_details/train_data_scaled.csv")
y_train = pd.read_csv("../data_details/y_train.csv", header=None, squeeze=True)  # squeeze=True converts it to a Series

# Split the data into training and validation sets (80% train, 20% validation)
X_train, X_val, y_train_split, y_val = train_test_split(X_train_scaled, y_train, test_size=0.2, random_state=42)

X_train.shape, X_val.shape, y_train_split.shape, y_val.shape


C:\Users\eseoghene\AppData\Local\Temp\ipykernel_5896\1762555031.py:6: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  y_train = pd.read_csv("../data_details/y_train.csv", header=None, squeeze=True)  # squeeze=True converts it to a Series


ValueError: Found input variables with inconsistent numbers of samples: [1460, 1461]

there is an error because there is a mismatch between the number of samples in the feature set (X_train_scaled) and the target variable (y_train). The feature set has 1460 samples, while the target variable has 1461.

 I'll check the first few rows of each dataset to see if we can identify the issue.

In [2]:
# Displaying the first few rows of X_train_scaled and y_train to inspect the data
X_train_scaled.head(), y_train.head()


(         Id  MSSubClass  LotFrontage   LotArea  OverallQual  OverallCond  \
 0 -1.730865    0.073375    -0.231877 -0.207142     0.651479    -0.517200   
 1 -1.728492   -0.872563     0.437043 -0.091886    -0.071836     2.179628   
 2 -1.726120    0.073375    -0.098093  0.073480     0.651479    -0.517200   
 3 -1.723747    0.309859    -0.454850 -0.096897     0.651479    -0.517200   
 4 -1.721374    0.073375     0.615421  0.375148     1.374795    -0.517200   
 
    YearBuilt  YearRemodAdd  MasVnrArea  BsmtFinSF1  ...  SaleCondition_Family  \
 0   1.050994      0.878668    0.514104    0.575425  ...             -0.117851   
 1   0.156734     -0.429577   -0.570750    1.171992  ...             -0.117851   
 2   0.984752      0.830215    0.325915    0.092907  ...             -0.117851   
 3  -1.863632     -0.720298   -0.570750   -0.499274  ...             -0.117851   
 4   0.951632      0.733308    1.366489    0.463568  ...             -0.117851   
 
    SaleCondition_Normal  SaleCondition_Pa

so it appears that the y_train dataset has an additional header row with the label "SalePrice." i think this is what is causing the mismatch in the number of samples.

I'll remove this header row from y_train and then check if it works 


In [3]:
# Removing the header row from y_train
y_train = y_train.iloc[1:].astype(int)

# Split the data into training and validation sets again
X_train, X_val, y_train_split, y_val = train_test_split(X_train_scaled, y_train, test_size=0.2, random_state=42)

X_train.shape, X_val.shape, y_train_split.shape, y_val.shape


((1168, 267), (292, 267), (1168,), (292,))

The data is now properly split into training and validation sets. We have 1168 samples in the training set and 292 samples in the validation set.

Next we can move on to training our regression models:

- Linear Regression
- Random Forest
- Gradient Boosting (using XGBoost)


For each model, I plan to:

Train the model using the training set.
Evaluate the model using the validation set.
Record the Root Mean Squared Error (RMSE) for each model as it's a common metric for regression problems and it's particularly relevant for our dataset since the Kaggle competition also uses RMSE.

first linear regression

In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Initialize and train the Linear Regression model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train_split)

# Predict on the validation set
lr_val_predictions = lr_model.predict(X_val)

# Calculate the RMSE for Linear Regression
lr_rmse = np.sqrt(mean_squared_error(y_val, lr_val_predictions))
lr_rmse


1093041192553732.4

then random forest

In [5]:
from sklearn.ensemble import RandomForestRegressor

# Initialize and train the Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train_split)

# Predict on the validation set
rf_val_predictions = rf_model.predict(X_val)

# Calculate the RMSE for Random Forest
rf_rmse = np.sqrt(mean_squared_error(y_val, rf_val_predictions))
rf_rmse


30447.655239822787

then gradient boosting

In [6]:
import xgboost as xgb

# Initialize and train the XGBoost model
xgb_model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75, colsample_bytree=1, max_depth=7, random_state=42)
xgb_model.fit(X_train, y_train_split)

# Predict on the validation set
xgb_val_predictions = xgb_model.predict(X_val)

# Calculate the RMSE for XGBoost
xgb_rmse = np.sqrt(mean_squared_error(y_val, xgb_val_predictions))
xgb_rmse


ModuleNotFoundError: No module named 'xgboost'